In [4]:
%%writefile program.cpp
#include<iostream>
using namespace std;
int main(){
  cout << "HELLO" << endl;
  return 0;
}

Writing program.cpp


In [5]:
!g++ program.cpp -o program.exe

In [6]:
!./program.exe

HELLO


#Home Task: 2D Convolution

In [38]:
%%writefile hometask.cu
#include <cuda.h>
#include <stdio.h>
#include <iostream>

using namespace std;

__constant__ int k = 3;
__constant__ int filter[9];

__global__ void function(int *input, int *output, int n, int m){
  int globalIdx = blockIdx.x * blockDim.x + threadIdx.x;
  if(globalIdx < n * m){
    int res = 0;

    for(int i = max(0, globalIdx / m - 1); i <= min(n - 1, globalIdx / m + 1); i++){
      for(int j = max(0, globalIdx % m - 1); j <= min(m - 1, globalIdx % m + 1); j++){
        int kx = (i - globalIdx / m) + 1;
        int ky = (j - globalIdx % m) + 1;
        res += input[i * m + j] * filter[kx * k + ky];
      }
    }

    output[globalIdx] = res;
  }
}

void print_matrix(int *matrix, int n, int m){
  for(int i = 0; i < n; i++){
    for(int j = 0; j < m; j++){
      cout << matrix[i * m + j] << " ";
    } cout << endl;
  }
}

int main(){
  int n = 3;
  int m = 3;
  int nxm = n * m;
  int size = nxm * sizeof(int);
  int *host_input = (int *) malloc(size);
  int *host_output = (int *) malloc(size);
  int host_filter[9];
  for(int i = 0; i < 3 ; i++){
    int v = 1;
    for(int j = 0; j < 3 ; j++){
      host_filter[i * 3 + j] = v;
      v--;
    }
  }

  for(int i = 0; i < nxm; i++){
    host_input[i] = i + 1;
  }

  int *device_input;
  int *device_output;
  cudaMalloc(&device_input, size);
  cudaMalloc(&device_output, size);
  cudaMemcpyToSymbol(filter, host_filter, sizeof(host_filter));
  cudaMemcpy(device_input, host_input, size, cudaMemcpyHostToDevice);

  int blocksize = 256;
  int gridsize = (nxm + blocksize - 1) / blocksize;
  function<<<gridsize, blocksize>>>(device_input, device_output, n, m);
  cudaMemcpy(host_output, device_output, size, cudaMemcpyDeviceToHost);

  cout << "INPUT MATRIX: " << endl;
  print_matrix(host_input, n, m);

  cout << "FILTER: " << endl;
  print_matrix(host_filter, 3, 3);

  cout << "OUTPUT MATRIX: " << endl;
  print_matrix(host_output, n, m);
  return 0;
}

Overwriting hometask.cu


In [39]:
!nvcc hometask.cu -o hometask
!./hometask

INPUT MATRIX: 
1 2 3 
4 5 6 
7 8 9 
FILTER: 
1 0 -1 
1 0 -1 
1 0 -1 
OUTPUT MATRIX: 
-7 -4 7 
-15 -6 15 
-13 -4 13 


# Task 2: Sum of Array

In [ ]:
%%writefile task2.cu
#include <cuda.h>
#include <stdio.h>

__global__ void blockReduceKernel(int *input, int *partialSums, int n) {
    extern __shared__ int sharedData[]; // Shared memory for the block

    int tid = threadIdx.x;
    int globalIdx = blockIdx.x * blockDim.x + tid; // Global index in array

    // zero-padding if out of bounds
    sharedData[tid] = (globalIdx < n) ? input[globalIdx] : 0;
    __syncthreads();

    for (int stride = blockDim.x / 2; stride > 0; stride /= 2) {
        if (tid < stride) {
            sharedData[tid] += sharedData[tid + stride];
        }
        __syncthreads();
    }

    if (tid == 0) {
        partialSums[blockIdx.x] = sharedData[0];
    }
}

__global__ void gridReduceKernel(int *partialSums, int *result, int numBlocks) {
    extern __shared__ int sharedData[]; // Shared memory for the block

    int tid = threadIdx.x;

    sharedData[tid] = (tid < numBlocks) ? partialSums[tid] : 0;
    __syncthreads();

    for (int stride = blockDim.x / 2; stride > 0; stride /= 2) {
        if (tid < stride) {
            sharedData[tid] += sharedData[tid + stride];
        }
        __syncthreads();
    }

    if (tid == 0) {
        *result = sharedData[0];
    }
}

int main() {
    const int n = 1024;                  // Size of the input array
    int size = n * sizeof(int);          // Total memory size for the array
    int blockSize = 256;                 // Number of threads per block
    int numBlocks = (n + blockSize - 1) / blockSize; // Number of blocks
    int partialSize = numBlocks * sizeof(int);

    // Host memory allocation
    int *h_input = (int *)malloc(size);
    int *h_partialSums = (int *)malloc(partialSize);
    int h_result = 0;

    for (int i = 0; i < n; i++) {
        h_input[i] = i + 1;
    }

    // Device memory allocation
    int *d_input, *d_partialSums, *d_result;
    cudaMalloc(&d_input, size);
    cudaMalloc(&d_partialSums, partialSize);
    cudaMalloc(&d_result, sizeof(int));

    cudaMemcpy(d_input, h_input, size, cudaMemcpyHostToDevice);

    blockReduceKernel<<<numBlocks, blockSize, blockSize * sizeof(int)>>>(d_input, d_partialSums, n);

    // Launch the second kernel
    if (numBlocks > 1) {
        gridReduceKernel<<<1, blockSize, blockSize * sizeof(int)>>>(d_partialSums, d_result, numBlocks);
        cudaMemcpy(&h_result, d_result, sizeof(int), cudaMemcpyDeviceToHost);
    } else {
        // If only one block, copy the partial sum directly
        cudaMemcpy(&h_result, d_partialSums, sizeof(int), cudaMemcpyDeviceToHost);
    }

    // Print the result
    printf("Sum: %d\n", h_result);

    // Free memory
    free(h_input);
    free(h_partialSums);
    cudaFree(d_input);
    cudaFree(d_partialSums);
    cudaFree(d_result);

    return 0;
}


Overwriting task2.cu


In [ ]:
!nvcc task2.cu -o task2
!./task2

Sum: 524800


# Task 1: Tiled Matrix Multiplication

In [ ]:
%%writefile task1.cu
#include <cuda.h>
#include <stdio.h>

#define TILE_SIZE 2

__global__ void tiledMatrixMul(int* A, int* B, int* C, int N) {
    __shared__ int tileA[TILE_SIZE][TILE_SIZE];
    __shared__ int tileB[TILE_SIZE][TILE_SIZE];

    int row = blockIdx.y * TILE_SIZE + threadIdx.y;
    int col = blockIdx.x * TILE_SIZE + threadIdx.x;

    int tempSum = 0;

    for (int tileIdx = 0; tileIdx < N / TILE_SIZE; tileIdx++) {
        tileA[threadIdx.y][threadIdx.x] = A[row * N + tileIdx * TILE_SIZE + threadIdx.x];
        tileB[threadIdx.y][threadIdx.x] = B[(tileIdx * TILE_SIZE + threadIdx.y) * N + col];
        __syncthreads();

        for (int k = 0; k < TILE_SIZE; k++) {
            tempSum += tileA[threadIdx.y][k] * tileB[k][threadIdx.x];
        }
        __syncthreads();
    }

    if (row < N && col < N) {
        C[row * N + col] = tempSum;
    }
}

int main() {
    const int N = 4;
    int size = N * N * sizeof(int);

    int A[N][N] = {
        {1, 2, 3, 4},
        {5, 6, 7, 8},
        {9, 10, 11, 12},
        {13, 14, 15, 16}
    };

    int B[N][N] = {
        {1, 0, 0, 0},
        {0, 1, 0, 0},
        {0, 0, 1, 0},
        {0, 0, 0, 1}
    };

    int C[N][N] = {0};

    int *d_A, *d_B, *d_C;

    cudaMalloc(&d_A, size);
    cudaMalloc(&d_B, size);
    cudaMalloc(&d_C, size);

    cudaMemcpy(d_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, size, cudaMemcpyHostToDevice);

    dim3 blockDim(TILE_SIZE, TILE_SIZE);
    dim3 gridDim(N / TILE_SIZE, N / TILE_SIZE);

    tiledMatrixMul<<<gridDim, blockDim>>>(d_A, d_B, d_C, N);

    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    printf("Matrix C:\n");
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            printf("%d ", C[i][j]);
        }
        printf("\n");
    }

    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}


Writing task1.cu


In [ ]:
!nvcc task1.cu -o task1
!./task1

Matrix C:
1 2 3 4 
5 6 7 8 
9 10 11 12 
13 14 15 16 
